In [1]:
import h1st as h1
h1.init()

import pandas as pd
import numpy as np
import sklearn.metrics


In [2]:
from AutomotiveCybersecurity.models.msg_freq_event_detector import MsgFreqEventDetectorModel
m = MsgFreqEventDetectorModel()
data = m.load_data(num_files=1)

In [3]:
m.train(data)

100%|██████████| 5/5 [00:09<00:00,  1.99s/it]


In [4]:
m.stats

,SteeringAngle,CarSpeed,YawRate,Gx,Gy
count,1417.000000,1417.000000,1417.000000,1417.000000,1417.000000
mean,32.930134,16.465773,32.929428,32.929428,32.929428
std,0.443155,0.499003,0.443840,0.443840,0.443840
min,32.000000,16.000000,32.000000,32.000000,32.000000
25%,33.000000,16.000000,33.000000,33.000000,33.000000
50%,33.000000,16.000000,33.000000,33.000000,33.000000
75%,33.000000,17.000000,33.000000,33.000000,33.000000
max,34.000000,17.000000,34.000000,34.000000,34.000000


In [5]:
# Don't run automatically this easily overwite latest version in AHT's computer, 
# I need to use correct version in the tutorial notebooks
# m.persist()

In [6]:
from AutomotiveCybersecurity.graph import WindowGenerator

df = pd.read_parquet(data['attack_files'][0])
df.columns = ['Timestamp', 'Label', 'CarSpeed', 'SteeringAngle', 'YawRate', 'Gx', 'Gy']

graph = h1.Graph()
graph.start()\
     .add(WindowGenerator())\
     .add(m)
graph.end()
results = graph.predict({"df": df})

results["event_detection_results"]

[{'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 599.6105},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 600.0105},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 600.4105},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 600.8104999999999},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 601.2104999999999},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 601.6104999999999},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 602.0104999999999},
 {'SteeringAngle': 0,
  'WindowIn

In [7]:
from AutomotiveCybersecurity.models.gradient_boosting_msg_classifier import GradientBoostingMsgClassifierModel

In [8]:
m2 = GradientBoostingMsgClassifierModel()
data = m2.load_data(20)
prepared_data = m2.prep_data(data)

len train_attack_df = 94623
len test_attack_df = 97424


In [9]:
m2.train(prepared_data)

In [10]:
# Don't run automatically this easily overwite latest version in AHT's computer, 
# I need to use correct version in the tutorial notebooks
# m2.persist()

In [11]:
m2.evaluate(prepared_data)

[[80886  1674]
 [13931   933]]
Accuracy = 0.8398


In [12]:
m2.metrics['confusion_matrix']

array([[80886,  1674],
       [13931,   933]])

In [13]:
from AutomotiveCybersecurity.graph import WindowGenerator

class NoOp(h1.Action):
    def call(self, command, inputs):
        pass

graph = h1.Graph()
graph.start()\
     .add(WindowGenerator())\
     .add(h1.Decision(m, decision_field="WindowInAttack"))\
     .add(yes=m2,
          no=NoOp())
graph.end()

df = pd.read_csv(data['test_attack_files'][0])
df.columns = ['Timestamp', 'Label', 'CarSpeed', 'SteeringAngle', 'YawRate', 'Gx', 'Gy',]

results = graph.predict({"df": df})

print(sklearn.metrics.confusion_matrix(results['injection_window_results']["Label"] == "Tx", 
                                       results['injection_window_results']["MsgIsAttack"]))
print(sklearn.metrics.accuracy_score(results['injection_window_results']["Label"] == "Tx", 
                                       results['injection_window_results']["MsgIsAttack"]))

[[4123    0]
 [ 826    0]]
0.8330975954738331


In [14]:
att_wins = len([x for x in results["event_detection_results"] if x["WindowInAttack"]])
att_wins

26

In [15]:
assert len(results) > 0

In [16]:
from AutomotiveCybersecurity.util import evaluate_event_graph

evaluate_event_graph(graph, prepared_data['test_files'][-2:])

s3://arimo-pana-cyber/data/sda1/IoT_CyberSecurity/autocyber_team_data/13Prius/injection/test/public/Attack/YawRate/Attack_YR_Cycle_Add_right-turn_01_010_1to1_D_03.csv
s3://arimo-pana-cyber/data/sda1/IoT_CyberSecurity/autocyber_team_data/13Prius/injection/test/public/Attack/Gy/Attack_GL1Y_Cycle_Add_forward-mid_01_005_1to1_Cd_01.csv
s3://arimo-pana-cyber/data/sda1/IoT_CyberSecurity/autocyber_team_data/13Prius/injection/test/public/Attack/CarSpeed/Attack_SP1_Cycle_Add_departure_01_025_1to1_B_02.csv
s3://arimo-pana-cyber/data/sda1/IoT_CyberSecurity/autocyber_team_data/13Prius/injection/test/public/Attack/SteeringAngle/Attack_SSA_Cycle_Add_stopping_01_095_1to1_A_02.csv
s3://arimo-pana-cyber/data/sda1/IoT_CyberSecurity/autocyber_team_data/13Prius/injection/test/public/Attack/Gy/Attack_GL1Y_Cycle_Add_right-turn_01_000_1to1_B_05.csv
s3://arimo-pana-cyber/data/sda1/IoT_CyberSecurity/autocyber_team_data/13Prius/injection/test/public/Attack/CarSpeed/Attack_SP1_Cycle_Add_forward-high_02_075_1to1_C

(462, 0, 0, 520)